# Tekstklassifiseringsoppgave

Som nevnt tidligere, skal vi fokusere på en enkel tekstklassifiseringsoppgave basert på **AG_NEWS**-datasettet, der målet er å klassifisere nyhetsoverskrifter i én av fire kategorier: Verden, Sport, Næringsliv og Vitenskap/Teknologi.

## Datasettet

Dette datasettet er innebygd i [`torchtext`](https://github.com/pytorch/text)-modulen, så vi kan enkelt få tilgang til det.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Her inneholder `train_dataset` og `test_dataset` samlinger som returnerer par av etikett (nummer på klasse) og tekst henholdsvis, for eksempel:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Så, la oss skrive ut de første 10 nye overskriftene fra datasettet vårt:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Fordi datasett er iteratorer, hvis vi vil bruke dataene flere ganger må vi konvertere det til en liste:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenisering

Nå må vi konvertere tekst til **tall** som kan representeres som tensorer. Hvis vi ønsker representasjon på ordnivå, må vi gjøre to ting:
* bruke **tokenizer** for å dele opp tekst i **tokens**
* bygge et **ordforråd** av disse tokens.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Bag of Words tekstrepresentasjon

Fordi ord representerer mening, kan vi noen ganger forstå meningen med en tekst bare ved å se på de enkelte ordene, uavhengig av rekkefølgen i setningen. For eksempel, når vi klassifiserer nyheter, er ord som *vær*, *snø* sannsynligvis indikatorer på *værmelding*, mens ord som *aksjer*, *dollar* kan peke mot *finansnyheter*.

**Bag of Words** (BoW) vektorrepresentasjon er den mest brukte tradisjonelle vektorrepresentasjonen. Hvert ord er knyttet til en vektorindeks, og hvert element i vektoren inneholder antall forekomster av et ord i et gitt dokument.

![Bilde som viser hvordan en bag of words-vektorrepresentasjon lagres i minnet.](../../../../../translated_images/no/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Merk**: Du kan også tenke på BoW som en sum av alle én-hot-kodede vektorer for de individuelle ordene i teksten.

Nedenfor er et eksempel på hvordan man kan generere en bag of words-representasjon ved hjelp av Scikit Learn Python-biblioteket:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

For å beregne bag-of-words-vektor fra vektorrepresentasjonen av vårt AG_NEWS-datasett, kan vi bruke følgende funksjon:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Merk:** Her bruker vi den globale variabelen `vocab_size` for å spesifisere standardstørrelsen på ordforrådet. Siden ordforrådsstørrelsen ofte er ganske stor, kan vi begrense størrelsen på ordforrådet til de mest brukte ordene. Prøv å redusere verdien av `vocab_size` og kjøre koden nedenfor, og se hvordan det påvirker nøyaktigheten. Du bør forvente et visst fall i nøyaktighet, men ikke dramatisk, til fordel for høyere ytelse.


## Trening av BoW-klassifiserer

Nå som vi har lært hvordan vi bygger Bag-of-Words-representasjon av teksten vår, la oss trene en klassifiserer basert på den. Først må vi konvertere datasettet vårt for trening på en slik måte at alle posisjonsvektorrepresentasjoner blir konvertert til Bag-of-Words-representasjon. Dette kan oppnås ved å sende `bowify`-funksjonen som `collate_fn`-parameter til standard torch `DataLoader`:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

La oss nå definere et enkelt klassifiserings-nevralt nettverk som inneholder ett lineært lag. Størrelsen på inputvektoren er lik `vocab_size`, og outputstørrelsen tilsvarer antall klasser (4). Fordi vi løser en klassifiseringsoppgave, er den endelige aktiveringsfunksjonen `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Nå skal vi definere standard treningssløyfe i PyTorch. Fordi datasettet vårt er ganske stort, vil vi for undervisningsformål kun trene i én epoke, og noen ganger til og med mindre enn én epoke (spesifisering av parameteren `epoch_size` lar oss begrense treningen). Vi vil også rapportere akkumulert treningsnøyaktighet under treningen; frekvensen for rapportering spesifiseres ved hjelp av parameteren `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrammer, TriGrammer og N-Grammer

En begrensning med en bag-of-words-tilnærming er at noen ord er en del av flerspråklige uttrykk. For eksempel har ordet 'hot dog' en helt annen betydning enn ordene 'hot' og 'dog' i andre sammenhenger. Hvis vi alltid representerer ordene 'hot' og 'dog' med de samme vektorene, kan det forvirre modellen vår.

For å løse dette brukes ofte **N-gram-representasjoner** i metoder for dokumentklassifisering, der frekvensen av hvert enkeltord, to-ordsuttrykk eller tre-ordsuttrykk er en nyttig funksjon for å trene klassifikatorer. I en bigram-representasjon, for eksempel, vil vi legge til alle ordpar i vokabularet, i tillegg til de opprinnelige ordene.

Nedenfor er et eksempel på hvordan man kan generere en bigram bag-of-words-representasjon ved hjelp av Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Den største ulempen med N-gram-tilnærmingen er at vokabularstørrelsen begynner å vokse ekstremt raskt. I praksis må vi kombinere N-gram-representasjon med noen teknikker for dimensjonsreduksjon, som *embeddings*, som vi skal diskutere i neste enhet.

For å bruke N-gram-representasjon i vårt **AG News**-datasett, må vi bygge et spesielt ngram-vokabular:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Vi kunne deretter bruke den samme koden som ovenfor for å trene klassifisereren, men det ville være svært minne-ineffektivt. I neste enhet vil vi trene bigram-klassifisereren ved hjelp av embeddings.

> **Note:** Du kan kun beholde de ngrammene som forekommer i teksten mer enn et spesifisert antall ganger. Dette vil sørge for at sjeldne bigrammer blir utelatt, og vil redusere dimensjonaliteten betydelig. For å gjøre dette, sett `min_freq`-parameteren til en høyere verdi, og observer hvordan vokabularlengden endrer seg.


## Termfrekvens Invers Dokumentfrekvens (TF-IDF)

I BoW-representasjon blir ordforekomster vektet likt, uavhengig av selve ordet. Det er imidlertid tydelig at hyppige ord, som *en*, *i*, osv., er langt mindre viktige for klassifisering enn spesialiserte termer. Faktisk er noen ord mer relevante enn andre i de fleste NLP-oppgaver.

**TF-IDF** står for **termfrekvens–invers dokumentfrekvens**. Det er en variant av bag of words, der man i stedet for en binær 0/1-verdi som indikerer tilstedeværelsen av et ord i et dokument, bruker en flyttallsverdi som er relatert til hvor ofte ordet forekommer i korpuset.

Mer formelt er vekten $w_{ij}$ til et ord $i$ i dokumentet $j$ definert som:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
hvor
* $tf_{ij}$ er antall forekomster av $i$ i $j$, altså BoW-verdien vi har sett tidligere
* $N$ er antall dokumenter i samlingen
* $df_i$ er antall dokumenter som inneholder ordet $i$ i hele samlingen

TF-IDF-verdien $w_{ij}$ øker proporsjonalt med antall ganger et ord vises i et dokument og justeres ned basert på antall dokumenter i korpuset som inneholder ordet. Dette hjelper med å kompensere for det faktum at noen ord forekommer oftere enn andre. For eksempel, hvis ordet vises i *alle* dokumentene i samlingen, er $df_i=N$, og $w_{ij}=0$, og disse termene vil bli fullstendig ignorert.

Du kan enkelt lage TF-IDF-vektorisering av tekst ved hjelp av Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Konklusjon

Selv om TF-IDF-representasjoner gir frekvensvekt til ulike ord, er de ikke i stand til å representere mening eller rekkefølge. Som den kjente lingvisten J. R. Firth sa i 1935: "Den fullstendige betydningen av et ord er alltid kontekstuelt, og ingen studie av betydning utenfor kontekst kan tas seriøst." Senere i kurset vil vi lære hvordan vi kan fange opp kontekstuell informasjon fra tekst ved hjelp av språkmodellering.



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
